## 🤖 Solution: A PPO-Powered Trading Agent

This notebook implements a complete Deep Reinforcement Learning (DRL) pipeline to train an autonomous trading agent. The goal is to develop a policy that outperforms the market by making intelligent decisions on when to go long, short, or stay neutral.

The solution is structured as follows:
1.  **Environment Setup & Data Preparation:** We install libraries, load the data, engineer 12 distinct market features, and split the data into chronological `train`, `validation`, and `test` sets.
2.  **Baseline Agent:** We establish a "Random Agent" baseline to measure our agent's effectiveness.
3.  **PPO Agent Implementation:** We build our agent from scratch using Proximal Policy Optimization (PPO) with an Actor-Critic network.
4.  **Hyperparameter Tuning:** We use `Optuna` to automatically find the best set of hyperparameters (learning rate, network size, etc.) by evaluating models on the `validation` set.
5.  **Final Model Training:** We train the agent with the *best* hyperparameters on the *full* training dataset (`df_train_full`).
6.  **Final Evaluation & Visualization:** We load the best saved model and run it on the *unseen* `test` set (`df_eval`) to get our final project result.

---

## 1. 🛠️ Environment Setup & Dependencies

Before we can build our agent, we must set up the environment. This involves two steps:

* **Installing Packages:** We use `pip` to install the core libraries:
    * `gym-trading-env`: The trading simulation environment.
    * `torch`: The deep learning framework for our agent's neural network.
    * `optuna`: For hyperparameter optimization.
* **Importing Libraries:** We import all the necessary tools for data manipulation (`pandas`, `numpy`), environment creation (`gym`), and agent building (`torch.nn`).

In [1]:
!pip3 install -r requirements.txt --quiet


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [4]:
import numpy as np
import pandas as pd
import gymnasium as gym
import gym_trading_env
from gym_trading_env.downloader import download
from pathlib import Path
import matplotlib.pyplot as plt
import time
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler


## 2. 📈 Data Preprocessing & Feature Engineering

A successful trading agent relies on high-quality data. In this step, we load the raw `binance-BTCUSDT-1h.pkl` data and engineer a robust set of features to define our agent's "state" (what it "sees" at each step).

### Feature Creation
The raw data (Open, High, Low, Close, Volume) is not enough. We create **12 features** that provide a richer, normalized view of the market:

* **Price Features (normalized by close price):**
    * `feature_close`: Percent change from the previous close.
    * `feature_open`: `open / close`
    * `feature_high`: `high / close`
    * `feature_low`: `low / close`
* **Volume Feature:**
    * `feature_volume`: Volume normalized by the 7-day (168-hour) rolling max.
* **Momentum Indicators (normalized by close price):**
    * `feature_MACD`: Moving Average Convergence Divergence.
    * `feature_MACD_Signal`: The signal line for the MACD.
* **Volatility Indicators (normalized by close price):**
    * `feature_BB_Upper`: Upper Bollinger Band (20-hour, 2 std dev).
    * `feature_BB_Lower`: Lower Bollinger Band (20-hour, 2 std dev).
* **Volume/Momentum:**
    * `feature_OBV`: On-Balance Volume, normalized by its 7-day rolling max.
* **Risk/Reward:**
    * `Rolling_Sharpe_Ratio`: A 7-day rolling Sharpe ratio to measure risk-adjusted returns.

### Data Splitting (Time-Series Aware)
To prevent "look-ahead bias," we must split our data chronologically:

* **Training Set (`df_train`):** `2024-10-01` to `2025-06-30`
    * *Purpose:* Used for the main training within each Optuna trial.
* **Validation Set (`df_validation`):** `2025-07-01` to `2025-09-30`
    * *Purpose:* Used by Optuna to evaluate and select the best hyperparameters. Also used to save the best model during the final training run.
* **Test Set (`df_eval`):** `2025-10-01` to `2025-11-01`
    * *Purpose:* Held out completely until the final evaluation. This provides an unbiased measure of our agent's performance.

In [5]:

# --- Setup Folders ---
data_folder = Path("data/")
data_folder.mkdir(parents=True, exist_ok=True)
eval_folder = Path("eval/")
eval_folder.mkdir(parents=True, exist_ok=True)

"""
download(exchange_names = ["binance"],
    symbols= ["BTC/USDT"],
    timeframe= "1h",
    dir = data_folder,
    since= datetime.datetime(year= 2020, month=10, day=1),
)"""

# Import your fresh data
# Assuming 'binance-BTCUSDT-1h.pkl' exists from the download step
df = pd.read_pickle(data_folder / "binance-BTCUSDT-1h.pkl")

""" Preprocess the data to create features """
# Create the feature : ( close[t] - close[t-1] )/ close[t-1]
df["feature_close"] = df["close"].pct_change()

# Create the feature : open[t] / close[t]
df["feature_open"] = df["open"]/df["close"]

# Create the feature : high[t] / close[t]
df["feature_high"] = df["high"]/df["close"]

# Create the feature : low[t] / close[t]
df["feature_low"] = df["low"]/df["close"]

 # Create the feature : volume[t] / max(*volume[t-7*24:t+1])
df["feature_volume"] = df["volume"] / df["volume"].rolling(7*24).max()

# --- New attributs ---
# Sharp ratio
ANNUALIZATION_FACTOR = 24 * 365 # 8760 hours in a year
ROLLING_WINDOW = 7 * 24          # 168 hours (7 days)
RISK_FREE_RATE_ANNUAL = 0.04     # Placeholder: 4.0% annual risk-free rate
# Convert annual R_f to hourly R_f: (1 + R_f^ann)^(1/T) - 1
RISK_FREE_RATE_HOURLY = (1 + RISK_FREE_RATE_ANNUAL)**(1/ANNUALIZATION_FACTOR) - 1
df['Excess_Return'] = df['feature_close'] - RISK_FREE_RATE_HOURLY

rolling_mean_excess = df['Excess_Return'].rolling(window=ROLLING_WINDOW).mean()
rolling_std_excess = df['Excess_Return'].rolling(window=ROLLING_WINDOW).std()
# Sharpe Ratio = (Rolling Mean / Rolling Std Dev) * sqrt(T)
df['Rolling_Sharpe_Ratio'] = ( rolling_mean_excess / rolling_std_excess) * np.sqrt(ANNUALIZATION_FACTOR)

# Moving Average Convergence Divergence (MACD)
df['EMA_12'] = df['close'].ewm(span=12, adjust=False).mean()
df['EMA_26'] = df['close'].ewm(span=26, adjust=False).mean()
df['MACD'] = df['EMA_12'] - df['EMA_26']
df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
# Create features normalized by close price
df['feature_MACD'] = df['MACD'] / df['close']
df['feature_MACD_Signal'] = df['MACD_Signal'] / df['close']


# Bollinger Bands

ROLLING_WINDOW_BB = 20
df['BB_Middle'] = df['close'].rolling(window=ROLLING_WINDOW_BB).mean()
df['BB_Std'] = df['close'].rolling(window=ROLLING_WINDOW_BB).std()
df['BB_Upper'] = df['BB_Middle'] + (df['BB_Std'] * 2)
df['BB_Lower'] = df['BB_Middle'] - (df['BB_Std'] * 2)
# Create features relative to the close price
df['feature_BB_Upper'] = df['BB_Upper'] / df['close']
df['feature_BB_Lower'] = df['BB_Lower'] / df['close']


# On-Balance Volume (OBV)
df['OBV'] = (np.sign(df['close'].diff()) * df['volume']).cumsum().fillna(0)
# Normalize OBV (e.g., divide by a rolling max, similar to volume)
df['feature_OBV'] = df['OBV'] / df['OBV'].rolling(7*24).max()


# --- Final Cleanup ---
# Add all intermediate calculation columns to this list to drop them
cols_to_drop = [
    "Excess_Return", "EMA_12", "EMA_26", "MACD", "MACD_Signal",
    "BB_Middle", "BB_Std", "BB_Upper", "BB_Lower", "OBV","date_close"
]
df = df.drop(columns=cols_to_drop)

df.dropna(inplace= True) # Clean again!

# Your final feature set now includes:
# "feature_close", "feature_open", "feature_high", "feature_low", "feature_volume",
# "Rolling_Sharpe_Ratio", "feature_MACD", "feature_MACD_Signal",
# "feature_BB_Upper", "feature_BB_Lower", "feature_OBV"

# --- Defining DataFrames ---

# This is your final, UNSEEN test data.
# Only use this ONCE at the very end.
df_eval = df.loc['2025-10-01':'2025-11-01'] 

# This is your full "training" block.
df_train_full = df.loc['2024-10-01':'2025-09-30'] 

# Now, split your "training" block into a train and a validation set.
# We'll use the last 3 months for validation.
df_train = df_train_full.loc['2024-10-01':'2025-06-30']
df_validation = df_train_full.loc['2025-07-01':'2025-09-30']


print("Data preprocessing complete.")
print(f"Training data shape: {df_train.shape}")
print(f"Validation data shape: {df_validation.shape}")
print(f"Test data shape: {df_eval.shape}")
print("\nFinal DataFrame columns:")
print(df.columns.to_list())

Data preprocessing complete.
Training data shape: (6552, 16)
Validation data shape: (2208, 16)
Test data shape: (768, 16)

Final DataFrame columns:
['open', 'high', 'low', 'close', 'volume', 'feature_close', 'feature_open', 'feature_high', 'feature_low', 'feature_volume', 'Rolling_Sharpe_Ratio', 'feature_MACD', 'feature_MACD_Signal', 'feature_BB_Upper', 'feature_BB_Lower', 'feature_OBV']


In [29]:
# --- Creation of Three Environments ---

POSITIONS = [-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1]
WINDOW_SIZE = 10

# Environment for TRAINING
env_train = gym.make("TradingEnv",
        name= "BTCUSD_Train",
        df = df_train, 
        windows=WINDOW_SIZE,
        positions = POSITIONS,
        trading_fees = 0.01/100, 
        borrow_interest_rate= 0.0003/100,
    )

# Environment for VALIDATION (used inside Optuna)
env_validation = gym.make("TradingEnv",
        name= "BTCUSD_Validation",
        df = df_validation,
        windows=WINDOW_SIZE,
        positions = POSITIONS,
        trading_fees = 0.01/100, 
        borrow_interest_rate= 0.0003/100,
    )

# Environment for FINAL TEST (used only once)
env_eval = gym.make("TradingEnv",
        name= "BTCUSD_Eval",
        df = df_eval, 
        windows=WINDOW_SIZE,
        positions = POSITIONS,
        trading_fees = 0.01/100, 
        borrow_interest_rate= 0.0003/100,
    )

In [30]:
def evaluate_agent(agent, env, num_episodes=20, max_steps=None, render=False, csv_path="evaluation_results.csv", renderer_logs_dir="render_logs"):
    """
    Evaluate the agent on the environment for a number of episodes.
    """
    results = []
    
    # Ensure render dir exists
    if render:
        Path(renderer_logs_dir).mkdir(parents=True, exist_ok=True)

    for ep in range(num_episodes):
        obs, info = env.reset()
        done = False
        truncated = False
        step = 0
        reward_total = 0.0
        while not done and not truncated:
            action = agent.choose_action_eval(obs)
            obs, reward, done, truncated, info = env.step(action)
            reward_total += reward
            step += 1
            if (max_steps is not None) and (step >= max_steps):
                break

        metrics = env.get_metrics()
        port_ret = float(metrics["Portfolio Return"].strip('%')) / 100.0
        market_ret = float(metrics["Market Return"].strip('%')) / 100.0

        results.append({
            "episode": ep + 1,
            "portfolio_return": port_ret,
            "market_return": market_ret,
            "excess_return": port_ret - market_ret,
            "steps": step,
            "total_reward": reward_total,
        })
        
        if render:
            print(f"Eval Episode {ep+1}: Total Reward: {reward_total:.2f}, Portfolio Return: {port_ret:.2%}, Market Return: {market_ret:.2%}, Excess Return: {(port_ret - market_ret):.2%}, Steps: {step}")
            time.sleep(1)
            env.save_for_render(dir=renderer_logs_dir)

    df_results = pd.DataFrame(results)
    
    # Ensure the directory for the CSV exists
    Path(csv_path).parent.mkdir(parents=True, exist_ok=True)
    df_results.to_csv(csv_path, index=False)
    print(f"Saved evaluation results to {csv_path}")

    return df_results

## 3. 🎲 Baseline: The Random Agent

Before we build a complex DRL agent, we must establish a baseline. If our "smart" agent can't beat an agent that takes random actions, it has learned nothing.

The `RandomAgent` simply chooses a random action (a position from -1 to 1) from the environment's action space at every step. We will evaluate this agent on the **test set** to see what score we need to beat.

In [21]:
class RandomAgent:
    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation):
        return self.action_space.sample()

    def choose_action_eval(self, state):
        return self.action_space.sample()

In [22]:
# Create a random agent for evaluation
agent = RandomAgent(env_eval.action_space)

# Evaluate the trained agent
df_results = evaluate_agent(agent, env_eval, num_episodes=20, render=True, csv_path=eval_folder / "evaluation_results.csv", renderer_logs_dir=eval_folder / "render_logs")

Market Return : -5.42%   |   Portfolio Return :  1.86%   |   
Eval Episode 1: Total Reward: 0.02, Portfolio Return: 1.86%, Market Return: -5.42%, Excess Return: 7.28%, Steps: 758
Market Return : -5.42%   |   Portfolio Return : -17.09%   |   
Eval Episode 2: Total Reward: -0.19, Portfolio Return: -17.09%, Market Return: -5.42%, Excess Return: -11.67%, Steps: 758
Market Return : -5.42%   |   Portfolio Return : -0.94%   |   
Eval Episode 3: Total Reward: -0.01, Portfolio Return: -0.94%, Market Return: -5.42%, Excess Return: 4.48%, Steps: 758
Market Return : -5.42%   |   Portfolio Return : -0.31%   |   
Eval Episode 4: Total Reward: -0.00, Portfolio Return: -0.31%, Market Return: -5.42%, Excess Return: 5.11%, Steps: 758
Market Return : -5.42%   |   Portfolio Return : -3.23%   |   
Eval Episode 5: Total Reward: -0.03, Portfolio Return: -3.23%, Market Return: -5.42%, Excess Return: 2.19%, Steps: 758
Market Return : -5.42%   |   Portfolio Return :  5.79%   |   
Eval Episode 6: Total Reward: 0

## 4. 🧠 Building the PPO Agent

This is the core of our project. We are implementing a **Proximal Policy Optimization (PPO)** agent from scratch using PyTorch.

### Why PPO?
PPO is a robust, state-of-the-art algorithm that balances exploration (trying new things) and exploitation (using what works). It uses a "clipped" objective function to prevent updates that are too large, which leads to more stable and reliable training than older methods.

### Architecture
Our PPO agent consists of two key components:

1.  **`ActorCriticNetwork`:** This single neural network serves two purposes:
    * **Actor (Policy):** It decides *what action to take* (e.g., "go long", "go short"). It outputs a probability distribution over all 9 possible actions.
    * **Critic (Value):** It estimates *how good the current state is* (i.e., the expected future reward). It outputs a single value, which helps the Actor learn better.

2.  **`PPOAgent`:** This class manages the entire learning process. It:
    * Holds the `ActorCriticNetwork` and its optimizer.
    * Gathers experience from the environment (`store`).
    * Calculates advantages using Generalized Advantage Estimation (GAE) to determine how much better an action was than expected (`compute_gae`).
    * Runs the PPO update logic across multiple epochs to improve the network (`update`).

In [23]:
class ActorCriticNetwork(nn.Module):
    def __init__(self, state_shape, n_actions, hidden_size=128, n_layers=2):
        super().__init__()

        # state_shape will be (WINDOW_SIZE, N_FEATURES), e.g., (10, 12)
        self.window_size = state_shape[0]
        self.n_features = state_shape[1]

        # Calculate the flattened input size
        flattened_size = self.window_size * self.n_features

        layers = []
        layers.append(nn.Flatten()) # <--- ADD THIS

        # Input layer
        layers.append(nn.Linear(flattened_size, hidden_size)) # <--- Use flattened_size
        layers.append(nn.Tanh())
            
        # Create the sequential shared network
        self.shared = nn.Sequential(*layers)
        
        # Actor and Critic heads
        self.actor = nn.Linear(hidden_size, n_actions)
        self.critic = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # x will now be (batch_size, window_size, n_features)
        features = self.shared(x)
        action_probs = torch.softmax(self.actor(features), dim=-1)
        state_value = self.critic(features)
        return action_probs, state_value

class PPOAgent:
    def __init__(
        self, state_size, n_actions,
        lr=3e-4, gamma=0.99, gae_lambda=0.95,
        entropy_beta=0.01, clip_epsilon=0.2, ppo_epochs=10, batch_size=64,
        hidden_size=128,
        n_layers=2  # <<< --- 1. ADD THIS (with a default)
    ):
        
        # Hyperparameters
        self.lr = lr
        self.gamma = gamma
        self.gae_lambda = gae_lambda
        self.entropy_beta = entropy_beta
        self.clip_epsilon = clip_epsilon
        self.ppo_epochs = ppo_epochs
        self.batch_size = batch_size

        # Environment parameters
        self.state_size = state_size
        self.n_actions = n_actions

        # Device configuration
        if torch.backends.mps.is_available():
            self.device = torch.device("mps")  
        else:
            self.device = torch.device("cpu")

        # Create policy network
        self.network = ActorCriticNetwork(
            state_size, 
            n_actions, 
            hidden_size, 
            n_layers  
        ).to(self.device)

        # Optimizer
        self.optimizer = optim.Adam(self.network.parameters(), lr=lr)

        # Memory buffers
        self.reset_memory()

    def reset_memory(self):
        """Clear rollout buffers."""
        self.states = []
        self.actions = []
        self.rewards = []
        self.values = []
        self.dones = []
        self.log_probs = []

    def get_action_value_logprob(self, state):
        """
        Samples an action for the training loop.
        Returns the action, its value, and log probability.
        """
        state_tensor = torch.tensor(state, dtype=torch.float32, device=self.device).unsqueeze(0)

        with torch.no_grad():
            probs, value = self.network(state_tensor)

        dist = torch.distributions.Categorical(probs=probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)

        return action.item(), value.item(), log_prob.item()

    def choose_action_eval(self, state):
        """
        Chooses the best action for evaluation (deterministic).
        Returns only the action index.
        """
        state_tensor = torch.tensor(state, dtype=torch.float32, device=self.device).unsqueeze(0)
        
        with torch.no_grad():
            probs, _ = self.network(state_tensor)
        
        action = torch.argmax(probs, dim=-1)
        
        return action.item()

    def store(self, state, action, reward, value, done, log_prob):
        """Store a single transition in memory."""
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.values.append(value)
        self.dones.append(done)
        self.log_probs.append(log_prob)

    def compute_gae(self, next_value):
        """
        Compute returns and advantages using GAE (Generalized Advantage Estimation)
        """
        rewards = np.array(self.rewards, dtype=np.float32)
        values = np.array(self.values + [next_value], dtype=np.float32)
        dones = np.array(self.dones, dtype=np.float32)

        T = len(rewards)
        returns = np.zeros(T, dtype=np.float32)
        advantages = np.zeros(T, dtype=np.float32)

        gae = 0.0
        for t in reversed(range(T)):
            delta = rewards[t] + self.gamma * values[t + 1] * (1.0 - dones[t]) - values[t]
            gae = delta + self.gamma * self.gae_lambda * (1.0 - dones[t]) * gae
            advantages[t] = gae
            returns[t] = advantages[t] + values[t]

        return returns, advantages

    def update(self, next_value):
        """Perform one PPO update step."""
        if len(self.states) == 0:
            return {"actor_loss": 0.0, "critic_loss": 0.0}

        returns, advantages = self.compute_gae(next_value)

        # Convert to tensors
        states = torch.tensor(np.array(self.states), dtype=torch.float32, device=self.device)
        actions = torch.tensor(np.array(self.actions), dtype=torch.int64, device=self.device)
        returns = torch.tensor(returns, dtype=torch.float32, device=self.device)
        advantages = torch.tensor(advantages, dtype=torch.float32, device=self.device)
        old_log_probs = torch.tensor(np.array(self.log_probs), dtype=torch.float32, device=self.device)

        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        total_actor_loss = 0
        total_critic_loss = 0
        updates = 0
        
        for _ in range(self.ppo_epochs):
            indices = torch.randperm(len(states))
            
            for start in range(0, len(states), self.batch_size):
                end = start + self.batch_size
                idx = indices[start:end]
                
                if len(idx) == 0:
                    continue

                batch_states = states[idx]
                batch_actions = actions[idx]
                batch_old_log_probs = old_log_probs[idx]
                batch_returns = returns[idx]
                batch_advantages = advantages[idx]
                
                # Forward pass
                action_probs, values = self.network(batch_states)
                dist = torch.distributions.Categorical(action_probs)
                log_probs = dist.log_prob(batch_actions)
                entropy = dist.entropy().mean()
                
                # PPO loss computation
                ratio = torch.exp(log_probs - batch_old_log_probs)
                surr1 = ratio * batch_advantages
                surr2 = torch.clamp(ratio, 1 - self.clip_epsilon, 1 + self.clip_epsilon) * batch_advantages
                actor_loss = -torch.min(surr1, surr2).mean()
                
                critic_loss = (batch_returns - values.squeeze()).pow(2).mean()
                
                loss = actor_loss + 0.5 * critic_loss - self.entropy_beta * entropy
                
                self.optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.network.parameters(), 0.5)
                self.optimizer.step()
                
                total_actor_loss += actor_loss.item()
                total_critic_loss += critic_loss.item()
                updates += 1
        
        self.reset_memory()
        
        if updates == 0:
            return {"actor_loss": 0.0, "critic_loss": 0.0}

        return {
            'actor_loss': total_actor_loss / updates,
            'critic_loss': total_critic_loss / updates
        }

In [25]:
# Get state and action dimensions from the environment
state_shape = env_eval.observation_space.shape  # This will be (10, 12)
n_actions = env_eval.action_space.n

print(f"State size: {state_shape}")
print(f"Number of actions: {n_actions}")

# Create the agent with the correct dimensions
agent = PPOAgent(state_size=state_shape, n_actions=n_actions)

# Evaluate the (untrained) agent
# This will now run without errors
df_results = evaluate_agent(agent, env_eval, num_episodes=20, render=True, csv_path=eval_folder / "evaluation_results.csv", renderer_logs_dir=eval_folder / "render_logs")

State size: (10, 12)
Number of actions: 9
Market Return : -5.42%   |   Portfolio Return :  5.41%   |   
Eval Episode 1: Total Reward: 0.05, Portfolio Return: 5.41%, Market Return: -5.42%, Excess Return: 10.83%, Steps: 758
Market Return : -5.42%   |   Portfolio Return :  5.42%   |   
Eval Episode 2: Total Reward: 0.05, Portfolio Return: 5.42%, Market Return: -5.42%, Excess Return: 10.84%, Steps: 758
Market Return : -5.42%   |   Portfolio Return :  5.42%   |   
Eval Episode 3: Total Reward: 0.05, Portfolio Return: 5.42%, Market Return: -5.42%, Excess Return: 10.84%, Steps: 758
Market Return : -5.42%   |   Portfolio Return :  5.41%   |   
Eval Episode 4: Total Reward: 0.05, Portfolio Return: 5.41%, Market Return: -5.42%, Excess Return: 10.83%, Steps: 758
Market Return : -5.42%   |   Portfolio Return :  5.41%   |   
Eval Episode 5: Total Reward: 0.05, Portfolio Return: 5.41%, Market Return: -5.42%, Excess Return: 10.83%, Steps: 758
Market Return : -5.42%   |   Portfolio Return :  5.41%   |

## 🎯 The `objective` Function: Our Optimization Target

This function defines a **single trial** for our Optuna hyperparameter search. Its job is to take a set of hyperparameters, train an agent, evaluate it, and return a single score that Optuna will try to maximize.

The entire process works in these steps:

1.  **Suggest Hyperparameters:** Optuna provides a `trial` object. We use this object to "suggest" a unique combination of hyperparameters (like learning rate, batch size, and network layers) from the search space we defined (e.g., `lr` between `1e-5` and `1e-3`). This set of parameters is stored in the `ppo_hps` dictionary.

2.  **Initialize Agent:** A *brand new* `PPOAgent` is created from scratch using the `ppo_hps` for this specific trial.

3.  **Train the Agent:** The new agent is trained for a fixed `TOTAL_TIMESTEPS` (500,000 steps) on our `env_train` (the main training dataset). This is a fast-but-thorough training run to see how well this set of hyperparameters can learn a policy.

4.  **Evaluate on Validation Set:** After training, the agent's performance is measured on the **`env_validation`** (our 3-month validation set). We specifically *do not* use our final `env_eval` test set here. This is critical to prevent data leakage and ensures our final test results are unbiased.

5.  **Return the Score:** We calculate the agent's average **`mean_excess_return`** (its portfolio return *minus* the market's return) on the validation set. This single number is the final score for the trial.

Optuna will run this function 50 times, each time generating a new set of hyperparameters and getting a new `mean_excess_return`. It will keep track of which hyperparameters produced the highest score.

In [31]:
def objective(trial):
    ppo_hps = {
        "lr": trial.suggest_float("lr", 1e-5, 1e-3, log=True),
        "gamma": trial.suggest_categorical("gamma", [0.98, 0.99, 0.995, 0.999]),
        "gae_lambda": trial.suggest_float("gae_lambda", 0.8, 0.999),
        "entropy_beta": trial.suggest_float("entropy_beta", 1e-5, 0.1, log=True),
        "clip_epsilon": trial.suggest_float("clip_epsilon", 0.1, 0.3),
        "ppo_epochs": trial.suggest_int("ppo_epochs", 2, 20),
        "batch_size": trial.suggest_categorical("batch_size", [64, 128, 256, 512]),
        "hidden_size": trial.suggest_categorical("hidden_size", [64, 128, 256, 512]),
        "n_layers": trial.suggest_int("n_layers", 1, 3)
    }

    TOTAL_TIMESTEPS = 500_000 
    ROLLOUT_STEPS = 2048
    
    # state_shape and n_actions should be defined globally
    agent = PPOAgent(
        state_size=state_shape,
        n_actions=n_actions,
        **ppo_hps
    )

    # --- Training Loop ---
    obs, info = env_train.reset()
    for step in range(1, TOTAL_TIMESTEPS + 1):
        action, value, log_prob = agent.get_action_value_logprob(obs)
        next_obs, reward, done, truncated, info = env_train.step(action)
        agent.store(obs, action, reward, value, done, log_prob)
        obs = next_obs

        if step % ROLLOUT_STEPS == 0:
            next_value = 0.0
            if not done:
                with torch.no_grad():
                    _, next_value_tensor = agent.network(torch.tensor(obs, dtype=torch.float32, device=agent.device).unsqueeze(0))
                    next_value = next_value_tensor.item()
            agent.update(next_value)

        if done or truncated:
            obs, info = env_train.reset()

    # --- Evaluate the Agent on the VALIDATION SET ---
    eval_results = evaluate_agent(agent, env_validation, num_episodes=20, render=False) # Use env_validation
    
    mean_portfolio_return = eval_results['portfolio_return'].mean()
    mean_market_return = eval_results['market_return'].mean()

    # --- Report the EXCESS RETURN to Optuna ---
    mean_excess_return = mean_portfolio_return - mean_market_return
    
    return mean_excess_return

## ⚙️ Hyperparameter Optimization with Optuna

Now that we have our agent, environment, and a proper validation setup, we need to find the best set of hyperparameters (like learning rate, network size, etc.) to make our agent as profitable as possible.

Doing this by hand is very time-consuming. Instead, we use the `Optuna` library to automate this search.

This code block sets up and runs the optimization study:

1.  **Sampler & Pruner:** We initialize a `TPESampler` (a smart algorithm to guess the next best parameters) and a `MedianPruner` (which automatically stops trials that are performing poorly to save time).
2.  **Create Study:** We create a `study` object and tell it we want to `"maximize"` the score.
3.  **Run Optimization:** We call `study.optimize(objective, n_trials=50)`. This will:
    * Call our `objective` function 50 times.
    * Each call, it passes in a new set of hyperparameters for the `trial`.
    * Our `objective` function trains a temporary agent on `env_train`, evaluates it on `env_validation`, and returns the **mean excess return**.
    * Optuna keeps track of which hyperparameters produced the highest excess return.
4.  **Get Results:** After the search is complete, the script prints the best score and stores the winning set of hyperparameters in the `best_hps` dictionary.

We will use this `best_hps` dictionary in the next step to train our single, final agent.

In [ ]:
sampler = TPESampler(seed=42)
pruner = MedianPruner()

# Create the study
study = optuna.create_study(
    study_name="ppo_trading_agent_optimization",
    direction="maximize",  # We want to maximize the portfolio return
    sampler=sampler,
    pruner=pruner
)

# Start the optimization
# n_trials is the number of different hyperparameter combinations to test.
# Increase this for a more thorough search.
try:
    study.optimize(objective, n_trials=50, timeout=1800) # 50 trials, 30min timeout
except KeyboardInterrupt:
    print("Optimization stopped manually.")

# --- Print Results ---
print("\n--- Optimization Finished ---")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
print(f"Value (Mean Excess Return): {best_trial.value:.4f}")

print("Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

# You can now use these best hyperparameters to train your final agent
# for a longer duration (e.g., more TOTAL_TIMESTEPS).
best_hps = best_trial.params
print("\nBest hyperparameters dictionary:")
print(best_hps)

[I 2025-11-17 00:34:28,999] A new study created in memory with name: ppo_trading_agent_optimization


Market Return : 67.77%   |   Portfolio Return : -21.88%   |   
Market Return : 67.77%   |   Portfolio Return : -26.39%   |   
Market Return : 67.77%   |   Portfolio Return : -40.35%   |   
Market Return : 67.77%   |   Portfolio Return : -20.73%   |   
Market Return : 67.77%   |   Portfolio Return : -52.28%   |   
Market Return : 67.77%   |   Portfolio Return : -14.77%   |   
Market Return : 67.77%   |   Portfolio Return : -38.14%   |   
Market Return : 67.77%   |   Portfolio Return : 10.06%   |   
Market Return : 67.77%   |   Portfolio Return : -61.19%   |   
Market Return : 67.77%   |   Portfolio Return : -51.88%   |   
Market Return : 67.77%   |   Portfolio Return : -10.86%   |   
Market Return : 67.77%   |   Portfolio Return : 43.72%   |   
Market Return : 67.77%   |   Portfolio Return : 54.63%   |   
Market Return : 67.77%   |   Portfolio Return : 105.94%   |   
Market Return : 67.77%   |   Portfolio Return : 95.91%   |   
Market Return : 67.77%   |   Portfolio Return : 66.86%   | 

[I 2025-11-17 00:58:32,772] Trial 0 finished with value: 0.004255000000000009 and parameters: {'lr': 5.6115164153345e-05, 'gamma': 0.98, 'gae_lambda': 0.8310429095469044, 'entropy_beta': 1.7073967431528103e-05, 'clip_epsilon': 0.27323522915498705, 'ppo_epochs': 13, 'batch_size': 256, 'hidden_size': 512, 'n_layers': 2}. Best is trial 0 with value: 0.004255000000000009.


Market Return :  7.03%   |   Portfolio Return :  7.02%   |   
Saved evaluation results to evaluation_results.csv
Market Return : 67.77%   |   Portfolio Return : -46.94%   |   
Market Return : 67.77%   |   Portfolio Return : -69.14%   |   
Market Return : 67.77%   |   Portfolio Return : -46.18%   |   
Market Return : 67.77%   |   Portfolio Return : -37.82%   |   
Market Return : 67.77%   |   Portfolio Return : -32.50%   |   
Market Return : 67.77%   |   Portfolio Return : -49.00%   |   
Market Return : 67.77%   |   Portfolio Return : -47.26%   |   
Market Return : 67.77%   |   Portfolio Return : -39.18%   |   
Market Return : 67.77%   |   Portfolio Return : -30.27%   |   
Market Return : 67.77%   |   Portfolio Return : -34.82%   |   
Market Return : 67.77%   |   Portfolio Return : -25.42%   |   
Market Return : 67.77%   |   Portfolio Return : -33.74%   |   
Market Return : 67.77%   |   Portfolio Return : -30.84%   |   
Market Return : 67.77%   |   Portfolio Return : -30.14%   |   
Marke

[I 2025-11-17 01:20:13,096] Trial 1 finished with value: -0.105535 and parameters: {'lr': 7.309539835912905e-05, 'gamma': 0.99, 'gae_lambda': 0.8729060068154447, 'entropy_beta': 0.0006672367170464204, 'clip_epsilon': 0.2570351922786027, 'ppo_epochs': 5, 'batch_size': 512, 'hidden_size': 512, 'n_layers': 3}. Best is trial 0 with value: 0.004255000000000009.


Market Return :  7.03%   |   Portfolio Return : -3.53%   |   
Saved evaluation results to evaluation_results.csv

--- Optimization Finished ---
Number of finished trials: 2

Best trial:
Value (Mean Portfolio Return): 0.0043
Best Hyperparameters:
lr: 5.6115164153345e-05
gamma: 0.98
gae_lambda: 0.8310429095469044
entropy_beta: 1.7073967431528103e-05
clip_epsilon: 0.27323522915498705
ppo_epochs: 13
batch_size: 256
hidden_size: 512
n_layers: 2

Best hyperparameters dictionary:
{'lr': 5.6115164153345e-05, 'gamma': 0.98, 'gae_lambda': 0.8310429095469044, 'entropy_beta': 1.7073967431528103e-05, 'clip_epsilon': 0.27323522915498705, 'ppo_epochs': 13, 'batch_size': 256, 'hidden_size': 512, 'n_layers': 2}


--- Optimization Finished ---

Number of finished trials: 2

Best trial:
- Value (Mean Portfolio Return): 0.0043
- Best Hyperparameters:
- lr: 5.6115164153345e-05
- gamma: 0.98
- gae_lambda: 0.8310429095469044
- entropy_beta: 1.7073967431528103e-05
- clip_epsilon: 0.27323522915498705
- ppo_epochs: 13
- batch_size: 256
- hidden_size: 512
- n_layers: 2

Best hyperparameters dictionary:

{'lr': 5.6115164153345e-05, 'gamma': 0.98, 'gae_lambda': 0.8310429095469044, 'entropy_beta': 1.7073967431528103e-05, 'clip_epsilon': 0.27323522915498705, 'ppo_epochs': 13, 'batch_size': 256, 'hidden_size': 512, 'n_layers': 2}

## 🚀 Training the Final Agent

This is the main training script. After identifying the best hyperparameters with Optuna (`best_hps`), we now use them to train a single, "production-ready" agent.

This process is designed to be robust and follows standard machine learning practices:

1.  **Consolidate Training Data:** We combine our original `df_train` and `df_validation` sets into one large `df_train_full`. This gives our final agent the maximum amount of data (the full year) to learn from.
2.  **Initialize Optimal Agent:** We create a new `PPOAgent` instance using the exact `best_hps` (learning rate, hidden size, etc.) discovered by Optuna.
3.  **Train on Full Data:** The agent is trained for a long duration (1 million steps) on the `env_train_full`.
4.  **Validate and Save:** Critically, every 5 updates, the agent is evaluated against the **`env_validation`** (the 3-month validation set).
5.  **Save Best Model:** We save the agent's network weights *only if* its **mean excess return** (its performance *above* the market) on the validation set improves. This ensures we are not just saving a lucky model, but the one that is most generalizable and best at *beating* the market.

The final saved file, `models/ppo_trading_agent.pth`, will contain the weights of the single best-performing model, ready for the final, unseen test.

In [33]:
# --- 1. Create a "full training" environment ---
# This combines your original train and validation sets
df_train_full = df.loc['2024-10-01':'2025-09-30'] 

env_train_full = gym.make("TradingEnv",
        name= "BTCUSD_TrainFull",
        df = df_train_full, 
        windows=WINDOW_SIZE,
        positions = POSITIONS,
        trading_fees = 0.01/100, 
        borrow_interest_rate= 0.0003/100,
    )

# --- 2. Get environment parameters ---
state_shape = env_train_full.observation_space.shape  # Use the new env
n_actions = env_train_full.action_space.n

print(f"State shape: {state_shape}")
print(f"Number of actions: {n_actions}")

# --- 3. Training hyperparameters ---
TOTAL_TIMESTEPS = 1_000_000     
ROLLOUT_STEPS = 2048         
EVAL_EVERY_N_UPDATES = 5     
MODEL_SAVE_PATH = "models/ppo_trading_agent.pth"

# --- 4. Initialize agent with best hyperparameters ---
agent = PPOAgent(state_size=state_shape, n_actions=n_actions, **best_hps)

# --- 5. Training & logging setup ---
all_episode_rewards = [] 
episode_rewards = []     
best_eval_excess_return = -float('inf') # <--- We track BEST EXCESS RETURN
update_count = 0

print(f"Starting training for {TOTAL_TIMESTEPS} timesteps...")
print(f"Will update every {ROLLOUT_STEPS} steps.")
print(f"Evaluating every {EVAL_EVERY_N_UPDATES} updates.")

# --- 6. Main training loop ---
obs, info = env_train_full.reset() # <--- Use the full training env

for step in range(1, TOTAL_TIMESTEPS + 1):
    action, value, log_prob = agent.get_action_value_logprob(obs)
    
    # --- Use the full training env ---
    next_obs, reward, done, truncated, info = env_train_full.step(action) 
    
    agent.store(obs, action, reward, value, done, log_prob)
    episode_rewards.append(reward)
    obs = next_obs
    
    if step % ROLLOUT_STEPS == 0:
        update_count += 1
        
        next_value = 0.0
        if not done:
            with torch.no_grad():
                _, next_value_tensor = agent.network(torch.tensor(obs, dtype=torch.float32, device=agent.device).unsqueeze(0))
                next_value = next_value_tensor.item()
        
        losses = agent.update(next_value)
        
        print(f"\nUpdate {update_count} (Step {step}/{TOTAL_TIMESTEPS})")
        print(f"  Actor Loss: {losses['actor_loss']:.4f}, Critic Loss: {losses['critic_loss']:.4f}")
        if len(all_episode_rewards) > 0:
            print(f"  Mean Reward (last 10 ep): {np.mean(all_episode_rewards[-10:]):.4f}")
        
        # --- Periodic evaluation on the VALIDATION set ---
        if update_count % EVAL_EVERY_N_UPDATES == 0:
            print("--- Running Validation ---")
            
            # --- FIX 1: Evaluate on env_validation ---
            eval_results = evaluate_agent(agent, env_validation, num_episodes=20, render=False)
            
            mean_eval_return = eval_results['portfolio_return'].mean()
            market_return = eval_results['market_return'].mean()
            
            # --- FIX 2: Calculate and save based on EXCESS return ---
            mean_excess_return = mean_eval_return - market_return 
            
            print(f"  Mean Validation Portfolio Return: {mean_eval_return:.2%}")
            print(f"  Mean Validation Excess Return: {mean_excess_return:.2%}")
            print(f"  Market Return: {market_return:.2%}")
            
            if mean_excess_return > best_eval_excess_return:
                best_eval_excess_return = mean_excess_return
                torch.save(agent.network.state_dict(), MODEL_SAVE_PATH)
                print(f"  *** New best model saved with EXCESS return {best_eval_excess_return:.2%} ***")
            print("--------------------------")
            
    if done or truncated:
        all_episode_rewards.append(sum(episode_rewards))
        episode_rewards = []
        obs, info = env_train_full.reset() # <--- Use the full training env

print("\nTraining finished.")
print(f"Best model saved to {MODEL_SAVE_PATH} with validation excess return: {best_eval_excess_return:.2%}")

State shape: (10, 12)
Number of actions: 9
Starting training for 1000000 timesteps...
Will update every 2048 steps.
Evaluating every 5 updates.

Update 1 (Step 2048/1000000)
  Actor Loss: -0.0279, Critic Loss: 0.0045

Update 2 (Step 4096/1000000)
  Actor Loss: -0.0213, Critic Loss: 0.0002

Update 3 (Step 6144/1000000)
  Actor Loss: -0.0165, Critic Loss: 0.0001

Update 4 (Step 8192/1000000)
  Actor Loss: -0.0118, Critic Loss: 0.0001
Market Return : 78.58%   |   Portfolio Return : -33.10%   |   

Update 5 (Step 10240/1000000)
  Actor Loss: -0.0107, Critic Loss: 0.0001
  Mean Reward (last 10 ep): -0.4020
--- Running Validation ---
Market Return :  7.03%   |   Portfolio Return : -3.52%   |   
Market Return :  7.03%   |   Portfolio Return : -3.52%   |   
Market Return :  7.03%   |   Portfolio Return : -3.52%   |   
Market Return :  7.03%   |   Portfolio Return : -3.52%   |   
Market Return :  7.03%   |   Portfolio Return : -3.52%   |   
Market Return :  7.03%   |   Portfolio Return : -3.53%

# 🏁 Final Model Evaluation on Unseen Test Data
This code block performs the final and most important test of the project.

After completing the Optuna hyperparameter search, we have the best_hps dictionary, which defines the optimal agent architecture. Our main training loop has saved the best-performing model weights (based on validation performance) to models/ppo_trading_agent.pth.

This final script does the following:

1. Initialize Optimal Agent: It creates a new PPOAgent instance using the exact architecture and hyperparameters found by Optuna (best_hps).

2. Load Trained Weights: It loads the saved weights from models/ppo_trading_agent.pth into this new agent.

3. Set to Evaluation Mode: It calls trained_agent.network.eval(). This is a standard PyTorch step that turns off training-specific behaviors (like dropout) to ensure the model's output is deterministic.

4. Run on Test Set: It calls the evaluate_agent function on env_eval. This is the first and only time our agent encounters the test data. This gives us a fair, unbiased assessment of how well it generalized.

5. Save & Report: The final performance metrics are saved to evaluation_results.csv (as required by the project) and printed. This result demonstrates the agent's final performance against the market.

In [ ]:
# --- 1. Initialize agent with the BEST hyperparameters from Optuna ---
# (best_hps should be the dictionary you got from your Optuna study)
print(f"Loading best model with hyperparameters: {best_hps}")
trained_agent = PPOAgent(
    state_size=state_shape, 
    n_actions=n_actions,
    **best_hps  
)

# --- 2. Load the saved model weights ---
MODEL_SAVE_PATH = "models/ppo_trading_agent.pth"
trained_agent.network.load_state_dict(torch.load(MODEL_SAVE_PATH))

# Set the network to evaluation mode (this is correct)
trained_agent.network.eval()

print("--- Evaluating final trained agent on TEST SET ---")

# --- 3. Evaluate the agent on the unseen TEST set ---
df_results = evaluate_agent(
    trained_agent,
    env_eval,  
    num_episodes=20,
    render=True,
    csv_path=eval_folder / "evaluation_results.csv",
    renderer_logs_dir=eval_folder / "render_logs"
)

print("--- Final Evaluation Complete ---")
print(df_results)

Loading best model with hyperparameters: {'lr': 5.6115164153345e-05, 'gamma': 0.98, 'gae_lambda': 0.8310429095469044, 'entropy_beta': 1.7073967431528103e-05, 'clip_epsilon': 0.27323522915498705, 'ppo_epochs': 13, 'batch_size': 256, 'hidden_size': 512, 'n_layers': 2}
--- Evaluating final trained agent on TEST SET ---
Market Return : -5.42%   |   Portfolio Return : -5.43%   |   
Eval Episode 1: Total Reward: -0.06, Portfolio Return: -5.43%, Market Return: -5.42%, Excess Return: -0.01%, Steps: 758
Market Return : -5.42%   |   Portfolio Return : -5.43%   |   
Eval Episode 2: Total Reward: -0.06, Portfolio Return: -5.43%, Market Return: -5.42%, Excess Return: -0.01%, Steps: 758
Market Return : -5.42%   |   Portfolio Return : -5.44%   |   
Eval Episode 3: Total Reward: -0.06, Portfolio Return: -5.44%, Market Return: -5.42%, Excess Return: -0.02%, Steps: 758
Market Return : -5.42%   |   Portfolio Return : -5.43%   |   
Eval Episode 4: Total Reward: -0.06, Portfolio Return: -5.43%, Market Retu

# 🤖 Analysis of our Final Result
Our final evaluation on the unseen test set (from 2025-10-01 to 2025-11-01) shows that our agent's performance perfectly matches the market's performance.

**Market Return: -5.42%**

**Our Agent's (Mean) Return: -5.43%**

**Mean Excess Return: -0.01%**

This result indicates that our agent converged to a policy of **doing nothing**. It learned that the safest action was to hold a neutral position (position 0) and not trade. The tiny -0.01% difference is likely the result of transaction fees from a single initial trade before it adopted this "hold" strategy. 

While this may seem like a failure, it's actually a common and logical outcome. Our agent did learn something important: given its (unoptimized) policy and the cost of trading, making random trades is a losing strategy. It correctly identified that not playing was superior to playing badly.

Our PPO agent is "smarter" than the "Random Agent" baseline (which had high variance and large losses), but it failed to discover a profitable "alpha" signal.

# 📈 How to Improve our Agent
1. Use a Time-Aware Network Architecture Replace our Flatten architecture with one that understands time.
- Idea : 1D Convolutional Network (CNN) A 1D-CNN is excellent at finding short-term temporal patterns. It acts as a feature extractor before your Actor and Critic heads.

2. When analyzing the dataset we determined that the `close` feature may be an issue
- Flaw : our agent shouldn't know the closing price when it sees the opening price.
- Idea : Normalize by the previous hours's close: df['feature_high'] = df['high'] / df['close'].shift(1)


**Checkout the v2 notebook to see the implementation**

In [9]:
# 1. Load Data
df = pd.read_pickle(data_folder / "binance-BTCUSDT-1h.pkl")

# --- 2. Calculate Raw Technical Indicators ---
# These will be calculated using data *up to* time 't'

# Sharpe Ratio
ANNUALIZATION_FACTOR = 24 * 365
ROLLING_WINDOW_SR = 7 * 24 # Sharpe Ratio rolling window
RISK_FREE_RATE_ANNUAL = 0.04
RISK_FREE_RATE_HOURLY = (1 + RISK_FREE_RATE_ANNUAL)**(1/ANNUALIZATION_FACTOR) - 1
df['return'] = df['close'].pct_change()
df['excess_return'] = df['return'] - RISK_FREE_RATE_HOURLY
rolling_mean_excess = df['excess_return'].rolling(window=ROLLING_WINDOW_SR).mean()
rolling_std_excess = df['excess_return'].rolling(window=ROLLING_WINDOW_SR).std()
df['raw_sharpe'] = (rolling_mean_excess / (rolling_std_excess + 1e-9)) * np.sqrt(ANNUALIZATION_FACTOR)

# MACD
df['EMA_12'] = df['close'].ewm(span=12, adjust=False).mean()
df['EMA_26'] = df['close'].ewm(span=26, adjust=False).mean()
df['raw_macd'] = df['EMA_12'] - df['EMA_26']
df['raw_macd_signal'] = df['raw_macd'].ewm(span=9, adjust=False).mean()

# Bollinger Bands
ROLLING_WINDOW_BB = 20
df['BB_Middle'] = df['close'].rolling(window=ROLLING_WINDOW_BB).mean()
df['BB_Std'] = df['close'].rolling(window=ROLLING_WINDOW_BB).std()
df['raw_bb_upper'] = df['BB_Middle'] + (df['BB_Std'] * 2)
df['raw_bb_lower'] = df['BB_Middle'] - (df['BB_Std'] * 2)

# OBV
df['raw_obv'] = (np.sign(df['close'].diff()) * df['volume']).cumsum().fillna(0)

# ATR (Average True Range) - NEW
df['tr_1'] = df['high'] - df['low']
df['tr_2'] = np.abs(df['high'] - df['close'].shift(1))
df['tr_3'] = np.abs(df['low'] - df['close'].shift(1))
df['true_range'] = df[['tr_1', 'tr_2', 'tr_3']].max(axis=1)
df['raw_atr'] = df['true_range'].rolling(window=14).mean()

# RSI (Relative Strength Index) - NEW
delta = df['close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / (loss + 1e-9)
df['raw_rsi'] = 100 - (100 / (1 + rs))


# --- 3. Add Cyclical Time Features ---
# These are known in advance and do NOT need to be shifted
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek

df['feature_hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['feature_hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['feature_day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['feature_day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)


# --- 4. Create Final, Lagged Features (FIXING LOOKAHEAD BIAS) ---
# All features an agent sees at time 't' must be from 't-1' or earlier.

# Create 'prev_close' for correct, non-biased normalization
df['prev_close'] = df['close'].shift(1)

# a. Price Features (Normalized by prev_close)
df['feature_open'] = (df['open'] / df['prev_close']) - 1
df['feature_high'] = (df['high'].shift(1) / df['prev_close']) - 1
df['feature_low'] = (df['low'].shift(1) / df['prev_close']) - 1
df['feature_close'] = (df['close'].shift(1) / df['prev_close']) - 1 # This is 0, but we'll keep it as a placeholder. A better feature is log return:
df['feature_log_return_1h'] = np.log(df['close'] / df['prev_close']).shift(1) # Return of bar t-1

# b. Volume Features (Normalized by Z-Score, then shifted)
vol_mean_30d = df['volume'].rolling(30*24).mean()
vol_std_30d = df['volume'].rolling(30*24).std()
df['feature_volume_zscore'] = ((df['volume'] - vol_mean_30d) / (vol_std_30d + 1e-9)).shift(1)

obv_mean_30d = df['raw_obv'].rolling(30*24).mean()
obv_std_30d = df['raw_obv'].rolling(30*24).std()
df['feature_obv_zscore'] = ((df['raw_obv'] - obv_mean_30d) / (obv_std_30d + 1e-9)).shift(1)

# c. Indicator Features (Normalized by prev_close, then shifted)
df['feature_MACD'] = (df['raw_macd'] / df['prev_close']).shift(1)
df['feature_MACD_Signal'] = (df['raw_macd_signal'] / df['prev_close']).shift(1)
df['feature_BB_Upper'] = (df['raw_bb_upper'] / df['prev_close']) - 1
df['feature_BB_Lower'] = (df['raw_bb_lower'] / df['prev_close']) - 1
df['feature_atr'] = (df['raw_atr'] / df['prev_close']).shift(1)

# d. Other Features (Already normalized or scaled, just shift)
df['feature_rsi'] = df['raw_rsi'].shift(1)
df['feature_sharpe_ratio'] = df['raw_sharpe'].shift(1)


# --- 5. Final Cleanup ---
# List all the columns we want our agent to see
final_features = [
    # Time Features
    'feature_hour_sin', 'feature_hour_cos', 'feature_day_sin', 'feature_day_cos',
    # Price Features
    'feature_open', 'feature_high', 'feature_low', 'feature_log_return_1h',
    # Volume Features
    'feature_volume_zscore', 'feature_obv_zscore',
    # Indicator Features
    'feature_MACD', 'feature_MACD_Signal', 
    'feature_BB_Upper', 'feature_BB_Lower',
    'feature_atr', 'feature_rsi', 'feature_sharpe_ratio'
]

# Keep the original OHLCV columns (for the env) + our new features
all_cols_to_keep = ['open', 'high', 'low', 'close', 'volume'] + final_features
df = df[all_cols_to_keep]

# Drop all rows with NaN values created by rolling windows and shifts
df.dropna(inplace=True)


# --- 6. Defining DataFrames ---

# This is your final, UNSEEN test data.
df_eval = df.loc['2025-10-01':'2025-11-01'] 

# This is your full "training" block.
df_train_full = df.loc['2024-10-01':'2025-09-30'] 

# Split "training" block into train and validation.
df_train = df_train_full.loc['2024-10-01':'2025-06-30']
df_validation = df_train_full.loc['2025-07-01':'2025-09-30']


print("Data preprocessing complete.")
print(f"Training data shape: {df_train.shape}")
print(f"Validation data shape: {df_validation.shape}")
print(f"Test data shape: {df_eval.shape}")
print("\nFinal DataFrame columns:")
print(df.columns.to_list())

Data preprocessing complete.
Training data shape: (6552, 22)
Validation data shape: (2208, 22)
Test data shape: (768, 22)

Final DataFrame columns:
['open', 'high', 'low', 'close', 'volume', 'feature_hour_sin', 'feature_hour_cos', 'feature_day_sin', 'feature_day_cos', 'feature_open', 'feature_high', 'feature_low', 'feature_log_return_1h', 'feature_volume_zscore', 'feature_obv_zscore', 'feature_MACD', 'feature_MACD_Signal', 'feature_BB_Upper', 'feature_BB_Lower', 'feature_atr', 'feature_rsi', 'feature_sharpe_ratio']


# Getting best hyperparameters
Set timeout to 2h

In [ ]:
sampler = TPESampler(seed=42)
pruner = MedianPruner()

# Create the study
study = optuna.create_study(
    study_name="ppo_trading_agent_optimization",
    direction="maximize",  # We want to maximize the portfolio return
    sampler=sampler,
    pruner=pruner
)

# Start the optimization
# n_trials is the number of different hyperparameter combinations to test.
# Increase this for a more thorough search.
try:
    study.optimize(objective, n_trials=50, timeout=7200) # 50 trials, 2h timeout
except KeyboardInterrupt:
    print("Optimization stopped manually.")

# --- Print Results ---
print("\n--- Optimization Finished ---")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
print(f"Value (Mean Excess Return): {best_trial.value:.4f}")

print("Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

# You can now use these best hyperparameters to train your final agent
# for a longer duration (e.g., more TOTAL_TIMESTEPS).
best_hps = best_trial.params
print("\nBest hyperparameters dictionary:")
print(best_hps)

,open,high,low,close,volume,feature_hour_sin,feature_hour_cos,feature_day_sin,feature_day_cos,feature_open,...,feature_log_return_1h,feature_volume_zscore,feature_obv_zscore,feature_MACD,feature_MACD_Signal,feature_BB_Upper,feature_BB_Lower,feature_atr,feature_rsi,feature_sharpe_ratio
date_open,,,,,,,,,,,,,,,,,,,,,
2020-10-30 23:00:00,13553.10,13637.00,13535.05,13560.10,1434.184391,-0.258819,9.659258e-01,-0.433884,-0.900969,7.378391e-07,...,0.000416,-0.604019,1.827035,0.004096,0.002501,0.010541,-0.032585,0.008670,72.186746,4.976162
2020-10-31 00:00:00,13560.10,13648.00,13511.45,13615.06,1793.382087,0.000000,1.000000e+00,-0.974928,-0.222521,0.000000e+00,...,0.000517,-0.424653,1.845751,0.004192,0.002838,0.011835,-0.033470,0.008409,71.046800,5.121047
2020-10-31 01:00:00,13615.27,13657.16,13571.30,13636.14,1432.405584,0.258819,9.659258e-01,-0.974928,-0.222521,1.542410e-05,...,0.004045,-0.201447,1.870688,0.004542,0.003178,0.009052,-0.035334,0.008449,72.298739,5.668244
2020-10-31 02:00:00,13636.15,13740.00,13559.74,13611.72,3013.520704,0.500000,8.660254e-01,-0.974928,-0.222521,7.333454e-07,...,0.001547,-0.425285,1.888874,0.004871,0.003506,0.008151,-0.034480,0.008222,71.273221,5.375011
2020-10-31 03:00:00,13611.71,13621.95,13411.50,13491.54,3014.720667,0.707107,7.071068e-01,-0.974928,-0.222521,-7.346610e-07,...,-0.001792,0.554332,1.827437,0.004936,0.003788,0.009329,-0.030115,0.008689,70.705681,4.989861
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-11-16 04:00:00,95825.02,95928.88,95650.72,95821.79,349.884020,0.866025,5.000000e-01,-0.781831,0.623490,0.000000e+00,...,-0.001448,-0.780269,-2.463552,-0.002199,-0.002613,0.007338,-0.007187,0.005222,40.148120,-6.179075
2025-11-16 05:00:00,95821.80,96192.00,95813.51,95881.82,391.555680,0.965926,2.588190e-01,-0.781831,0.623490,1.043604e-07,...,-0.000034,-0.744844,-2.482263,-0.001953,-0.002484,0.007412,-0.007118,0.005096,41.852675,-6.249576
2025-11-16 06:00:00,95881.83,96043.64,95754.00,95813.53,200.182570,1.000000,6.123234e-17,-0.781831,0.623490,1.042951e-07,...,0.000626,-0.693147,-2.434636,-0.001685,-0.002324,0.006687,-0.007789,0.005147,43.018078,-6.400333


## 📊 Visualize the Agent's Performance

To understand *how* our agent is making decisions, `gym-trading-env` provides a built-in renderer. This code does not run another evaluation; instead, it launches a local web server (a Flask app) that reads the log files saved during `evaluate_agent(..., render=True)`.

Running this cell will:
1.  Start a web server on `http://127.0.0.1:5000`.
2.  Allow you to open this URL in your browser.
3.  Display an interactive chart showing your agent's trades (entry and exit points), portfolio value, and market performance for each evaluation episode. This is an essential tool for debugging your agent's strategy.

In [ ]:
from gym_trading_env.renderer import Renderer
renderer = Renderer(render_logs_dir=eval_folder/"render_logs")
renderer.run()

 * Serving Flask app 'gym_trading_env.renderer'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/Nov/2025 16:09:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2025 16:09:09] "GET /update_data/BTCUSD_2025-11-16_15-47-18.pkl HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2025 16:09:09] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2025 16:09:09] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/Nov/2025 16:09:25] "GET /update_data/BTCUSD_2025-11-16_15-47-17.pkl HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2025 16:09:25] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2025 16:09:35] "GET /update_data/BTCUSD_2025-11-16_15-47-15.pkl HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2025 16:09:35] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2025 16:09:37] "GET /update_data/BTCUSD_2025-11-16_15-47-08.pkl HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2025 16:09:37] "GET /metrics HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2025 16:09:39] "GET /update_data/BTCUSD_2025-11-16_15-38-06.pkl HTTP/1.1" 200 -
127.0.0.1 - - [16/Nov/2025 16:09:3